# California Neighborhood Analysis

## Importing required Libraries

In [120]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Reading data from the processed file

In [6]:
neighborhoodsCalifornia = pd.read_csv('final_California_csv')

In [7]:
address = 'california'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of California are {}, {}.'.format(latitude, longitude))

c:\users\asif hashmi\appdata\local\programs\python\python36\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of California are 36.7014631, -118.7559974.


In [8]:
neighborhoodsCalifornia.drop(['Unnamed: 0'],axis=1,inplace = True)
neighborhoodsCalifornia.head()

,Neighborhood,Latitude,Longitude
0,Acampo,38.174640,-121.278559
1,Acton,34.480742,-118.186838
2,Adelaide,37.787483,-122.412150
3,Adelanto,34.582770,-117.409215
4,Adin,41.193780,-120.945245


## Displaying neighborhoods of California on map

In [10]:
neighborhoodCaliforniaFinal = neighborhoodsCalifornia.set_index('Neighborhood')

In [11]:
neighborhoodCaliforniaFinal.drop(['Madison','Hollister','Le Grand','Columbia','Boulevard','La Puente','Nuevo','Alameda','Vista'],inplace = True)

In [12]:
neighborhoodCaliforniaFinal.drop('Lincoln',inplace = True)
neighborhoodCaliforniaFinal.head()

,Latitude,Longitude
Neighborhood,,
Acampo,38.174640,-121.278559
Acton,34.480742,-118.186838
Adelaide,37.787483,-122.412150
Adelanto,34.582770,-117.409215
Adin,41.193780,-120.945245


In [13]:


map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoodCaliforniaFinal['Latitude'], neighborhoodCaliforniaFinal['Longitude'], neighborhoodCaliforniaFinal.index):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_newyork)  
    
map_newyork

## Using Foursquare API

In [14]:
CLIENT_ID = 'RNUKF33AES3WW3S5L4C1MYZUWPD0NH5PRWUFUKYCT535CJIC' # your Foursquare ID
CLIENT_SECRET = 'X1M3D0QK2GBTMUQNRCPWFFHM1EKOD4VQNSAND141PWGHDEVS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RNUKF33AES3WW3S5L4C1MYZUWPD0NH5PRWUFUKYCT535CJIC
CLIENT_SECRET:X1M3D0QK2GBTMUQNRCPWFFHM1EKOD4VQNSAND141PWGHDEVS


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
cal_venues2 = getNearbyVenues(names = neighborhoodsCalifornia['Neighborhood'][700:1156],
                                  latitudes = neighborhoodsCalifornia['Latitude'][700:1156],
                                  longitudes = neighborhoodsCalifornia['Longitude'][700:1156])

In [ ]:
neighborhoodsCalifornia['Neighborhood'][700:]

In [17]:
cal_venues = pd.read_csv('cal_venues_csv')

In [18]:
cal_venues2 = pd.read_csv('cal_venues2_csv')

In [19]:
cal_venues_final = cal_venues.append(cal_venues2)

In [20]:
cal_venues_final.shape

(13010, 8)

In [21]:
cal_venues_final.drop(['Unnamed: 0'],axis =1,inplace = True)

In [121]:
cal_venues_final.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acampo,38.174640,-121.278559,Dancing Coyote Wines,38.175776,-121.279119,Winery
1,Acampo,38.174640,-121.278559,XFINITY Store by Comcast,38.174479,-121.277508,Business Service
2,Adelaide,37.787483,-122.412150,The Olympic Club,37.788181,-122.411067,Gym / Fitness Center
3,Adelaide,37.787483,-122.412150,Ryoko's Japanese Restaurant & Bar,37.788183,-122.411882,Sushi Restaurant
4,Adelaide,37.787483,-122.412150,Wilson & Wilson Private Detective Agency,37.785969,-122.412969,Speakeasy


In [23]:
cal_venues_final.shape

(13010, 7)

In [ ]:
cal_venues_final.to_csv('cal_venues_final_csv')

## Converting the above table for clusternig process

In [90]:
# one hot encoding
cal_onehot = pd.get_dummies(cal_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cal_onehot['Neighborhood'] = cal_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cal_onehot.columns[-1]] + list(cal_onehot.columns[:-1])
cal_onehot = cal_onehot[fixed_columns]

cal_onehot.head()

,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,City Hall,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Hunting Supply,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lottery Retailer,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,Observatory,Office,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Public Bathroom,RV Park,Racetrack,Rafting,R

In [91]:
cal_onehot.shape

(7272, 391)

In [92]:
cal_grouped =cal_onehot.groupby('Neighborhood').mean().reset_index()
cal_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,City Hall,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Hunting Supply,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lottery Retailer,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,Observatory,Office,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Public Bathroom,RV Park,Racetrack,Rafting,R

In [93]:
cal_grouped.shape

(529, 391)

In [94]:
num_top_venues = 5

for hood in cal_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = cal_grouped[cal_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acampo----
              venue  freq
0  Business Service   0.5
1            Winery   0.5
2       Outlet Mall   0.0
3          Pharmacy   0.0
4         Pet Store   0.0


----Adelaide----
          venue  freq
0         Hotel  0.07
1  Cocktail Bar  0.06
2       Theater  0.04
3     Speakeasy  0.04
4           Bar  0.04


----Adelanto----
              venue  freq
0      Concert Hall   0.2
1              Food   0.2
2   Thai Restaurant   0.2
3            Lawyer   0.2
4  Business Service   0.2


----Adin----
                venue  freq
0              Market   0.5
1        Burger Joint   0.5
2         Yoga Studio   0.0
3                Park   0.0
4  Photography Studio   0.0


----Agoura Hills----
                           venue  freq
0                   Home Service   0.5
1                Thai Restaurant   0.5
2  Paper / Office Supplies Store   0.0
3                       Pharmacy   0.0
4                      Pet Store   0.0


----Agua Dulce----
                venue  freq
0  Mexican Res

               venue  freq
0            Brewery  0.14
1             Bakery  0.14
2        Coffee Shop  0.07
3  French Restaurant  0.07
4       Burger Joint  0.07


----Atherton----
                venue  freq
0      Baseball Field  0.17
1  Mexican Restaurant  0.17
2       Train Station  0.17
3   Food & Drink Shop  0.17
4                 Spa  0.17


----Atwater----
                  venue  freq
0   American Restaurant  0.33
1    Mexican Restaurant  0.17
2                 Diner  0.17
3  Fast Food Restaurant  0.17
4     Convenience Store  0.17


----Auburn----
               venue  freq
0        Coffee Shop  0.17
1            Brewery  0.08
2  Convenience Store  0.08
3         Steakhouse  0.08
4     Breakfast Spot  0.08


----Avalon----
                venue  freq
0               Hotel  0.21
1  Seafood Restaurant  0.09
2  Mexican Restaurant  0.06
3         Pizza Place  0.04
4       Grocery Store  0.04


----Avenal----
                       venue  freq
0         Mexican Restaurant  0.50
1 

                venue  freq
0       Grocery Store  0.29
1           Surf Spot  0.14
2               Beach  0.14
3               Hotel  0.14
4  Seafood Restaurant  0.14


----Bonita----
          venue  freq
0  Home Service   1.0
1          Pier   0.0
2      Pharmacy   0.0
3     Pet Store   0.0
4   Pet Service   0.0


----Bonsall----
         venue  freq
0      Stables   1.0
1  Yoga Studio   0.0
2  Outlet Mall   0.0
3    Pet Store   0.0
4  Pet Service   0.0


----Boonville----
                 venue  freq
0  American Restaurant  0.16
1        Grocery Store  0.11
2           Restaurant  0.11
3        Deli / Bodega  0.11
4         Dessert Shop  0.05


----Borrego Springs----
                           venue  freq
0                 Scenic Lookout   1.0
1                    Yoga Studio   0.0
2  Paper / Office Supplies Store   0.0
3                       Pharmacy   0.0
4                      Pet Store   0.0


----Boulevard----
              venue  freq
0  Business Service   0.5
1            

         venue  freq
0   Steakhouse   1.0
1  Yoga Studio   0.0
2  Outlet Mall   0.0
3    Pet Store   0.0
4  Pet Service   0.0


----Caspar----
                     venue  freq
0         Business Service   0.4
1  State / Provincial Park   0.2
2              Music Venue   0.2
3                    Hotel   0.2
4              Yoga Studio   0.0


----Cassel----
         venue  freq
0   Campground   1.0
1  Yoga Studio   0.0
2         Pier   0.0
3     Pharmacy   0.0
4    Pet Store   0.0


----Castaic----
       venue  freq
0      Diner  0.29
1     Bakery  0.14
2  Gastropub  0.14
3  Pet Store  0.14
4      Motel  0.14


----Castro Valley----
                  venue  freq
0        Sandwich Place  0.09
1                   Bar  0.09
2  Fast Food Restaurant  0.09
3     Korean Restaurant  0.06
4                 Hotel  0.06


----Castroville----
                venue  freq
0  Mexican Restaurant   0.2
1                Farm   0.2
2           BBQ Joint   0.2
3          Smoke Shop   0.2
4        Liquor St

               venue  freq
0               Park  0.09
1        Music Store  0.09
2                Bar  0.09
3   Sushi Restaurant  0.04
4  Convenience Store  0.04


----Coulterville----
                           venue  freq
0                            Pub  0.25
1                       Dive Bar  0.25
2                   Burger Joint  0.25
3                Bed & Breakfast  0.25
4  Paper / Office Supplies Store  0.00


----Courtland----
                           venue  freq
0                           Fair   0.5
1                  Grocery Store   0.5
2                    Yoga Studio   0.0
3  Paper / Office Supplies Store   0.0
4                       Pharmacy   0.0


----Covina----
                       venue  freq
0                        Bar  0.11
1             Cosmetics Shop  0.11
2  Middle Eastern Restaurant  0.11
3                       Food  0.11
4            Thai Restaurant  0.11


----Coyote----
                 venue  freq
0  Sporting Goods Shop   0.5
1                 Park   

...

            venue  freq
0  Ice Cream Shop  0.25
1       Disc Golf  0.25
2      Campground  0.25
3            Café  0.25
4     Yoga Studio  0.00


----Huntington Beach----
                venue  freq
0         Pizza Place  0.12
1  Mexican Restaurant  0.12
2                 Spa  0.06
3             Theater  0.06
4                Bank  0.06


----Huntington Park----
                venue  freq
0  Mexican Restaurant  0.10
1            Pharmacy  0.10
2   Convenience Store  0.10
3   Mobile Phone Shop  0.10
4         Coffee Shop  0.05


----Idyllwild----
                    venue  freq
0              Restaurant  0.29
1      Mexican Restaurant  0.14
2                    Café  0.14
3              Campground  0.14
4  Thrift / Vintage Store  0.14


----Imperial Beach----
                  venue  freq
0  Fast Food Restaurant  0.14
1                   Bar  0.09
2         Grocery Store  0.09
3         Deli / Bodega  0.05
4     Convenience Store  0.05


----Indian Wells----
           venue  freq
0   

            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.08
2    Home Service  0.08
3             ATM  0.08
4  Breakfast Spot  0.08


----La Verne----
                venue  freq
0  Italian Restaurant  0.06
1           Gastropub  0.06
2  Chinese Restaurant  0.06
3              Bistro  0.06
4      Sandwich Place  0.06


----Ladera Ranch----
         venue  freq
0         Park  0.50
1   Restaurant  0.25
2        Trail  0.25
3  Yoga Studio  0.00
4  Outlet Mall  0.00


----Laguna Beach----
                     venue  freq
0      American Restaurant  0.09
1       Seafood Restaurant  0.06
2       Italian Restaurant  0.03
3         Sushi Restaurant  0.03
4  New American Restaurant  0.03


----Laguna Hills----
                 venue  freq
0                 Park  0.38
1           Skate Park  0.12
2  American Restaurant  0.12
3              Theater  0.12
4           Playground  0.12


----Laguna Niguel----
                           venue  freq
0               Greek Restaurant  0.33


                venue  freq
0  Mexican Restaurant  0.14
1               Diner  0.09
2         Pizza Place  0.09
3   Korean Restaurant  0.09
4   Martial Arts Dojo  0.09


----Lompoc----
                 venue  freq
0          Coffee Shop  0.11
1   Mexican Restaurant  0.11
2  American Restaurant  0.11
3                  Pub  0.06
4       Farmers Market  0.06


----Long Barn----
          venue  freq
0         Hotel  0.33
1          Pool  0.33
2  Skating Rink  0.33
3   Yoga Studio  0.00
4          Park  0.00


----Long Beach----
                 venue  freq
0   Mexican Restaurant  0.12
1      Thai Restaurant  0.08
2          Coffee Shop  0.08
3  Arts & Crafts Store  0.08
4             Dive Bar  0.04


----Loomis----
                  venue  freq
0  Fast Food Restaurant  0.15
1            Restaurant  0.08
2        Sandwich Place  0.08
3          Burger Joint  0.08
4    Burmese Restaurant  0.08


----Los Alamitos----
                 venue  freq
0  American Restaurant  0.21
1               

                   venue  freq
0    Rental Car Location  0.11
1  Vietnamese Restaurant  0.11
2         Clothing Store  0.11
3          Hot Dog Joint  0.11
4             Shoe Store  0.11


----Mill Valley----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.05
2           Gift Shop  0.05
3         Pizza Place  0.05
4         Comedy Club  0.03


----Millbrae----
                   venue  freq
0     Chinese Restaurant  0.12
1            Coffee Shop  0.06
2  Vietnamese Restaurant  0.06
3                 Bakery  0.04
4           Dessert Shop  0.04


----Milpitas----
                   venue  freq
0  Vietnamese Restaurant  0.10
1      Indian Restaurant  0.10
2         Sandwich Place  0.08
3   Fast Food Restaurant  0.05
4        Bubble Tea Shop  0.05


----Mineral----
                           venue  freq
0                     Restaurant  0.33
1                Bed & Breakfast  0.33
2                          Motel  0.33
3                    Yoga Studio  0.00
4

           venue  freq
0  Deli / Bodega  0.27
1    Pizza Place  0.09
2          Plaza  0.09
3    Coffee Shop  0.09
4  Grocery Store  0.09


----Newhall----
                venue  freq
0  Mexican Restaurant  0.23
1             Theater  0.09
2      Breakfast Spot  0.09
3              Market  0.05
4       Train Station  0.05


----Newman----
            venue  freq
0     Pizza Place  0.18
1  Cosmetics Shop  0.09
2   Grocery Store  0.09
3            Food  0.09
4        Pharmacy  0.09


----Newport Beach----
                venue  freq
0         Coffee Shop  0.09
1  Seafood Restaurant  0.09
2  Mexican Restaurant  0.09
3      Sandwich Place  0.06
4          Restaurant  0.06


----Newport Coast----
        venue  freq
0        Café   0.2
1  Playground   0.2
2        Park   0.2
3      Church   0.2
4  Food Truck   0.2


----Nicasio----
                 venue  freq
0    Convenience Store   0.2
1                 Farm   0.2
2  American Restaurant   0.2
3       History Museum   0.2
4       Baseball

In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## List of Top 10 venues for every neighborhood

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_cal = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_cal['Neighborhood'] = cal_grouped['Neighborhood']

for ind in np.arange(cal_grouped.shape[0]):
    neighborhoods_venues_sorted_cal.iloc[ind, 1:] = return_most_common_venues(cal_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_cal.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acampo,Winery,Business Service,Women's Store,Food Service,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant
1,Adelaide,Hotel,Cocktail Bar,Bar,Theater,Speakeasy,Art Gallery,Thai Restaurant,Indian Restaurant,Hostel,Spa
2,Adelanto,Business Service,Lawyer,Thai Restaurant,Food,Concert Hall,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant
3,Adin,Burger Joint,Market,Women's Store,Food Service,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant
4,Agoura Hills,Home Service,Thai Restaurant,Women's Store,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant


In [98]:
cal_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,City Hall,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Hunting Supply,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Leather Goods Store,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lottery Retailer,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Night Market,Nightclub,Nightlife Spot,Noodle House,Observatory,Office,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Paper / Office Supplies Store,Park,Parking,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Print Shop,Pub,Public Art,Public Bathroom,RV Park,Racetrack,Rafting,R

In [30]:
cal_grouped.to_csv('clustering_data')

In [33]:
cal_grouped.shape

(529, 391)

## Applying Kmeans on the data

In [99]:
kclusters = 5

cal_grouped_clustering = cal_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cal_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 1, 1, 4, 1, 1, 1, 1, 1])

In [100]:
kmeans.labels_

array([3, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       4, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 4,
       1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1,
       1, 1, 1, 1, 1, 3, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

In [101]:
neighborhoods_venues_sorted_cal.shape

(529, 11)

In [102]:
neighborhood_merged = neighborhoods_venues_sorted_cal

# add clustering labels
neighborhood_merged['Cluster Labels'] = kmeans.labels_


In [103]:
neighborhood_merged.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Acampo,Winery,Business Service,Women's Store,Food Service,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,3
1,Adelaide,Hotel,Cocktail Bar,Bar,Theater,Speakeasy,Art Gallery,Thai Restaurant,Indian Restaurant,Hostel,Spa,1
2,Adelanto,Business Service,Lawyer,Thai Restaurant,Food,Concert Hall,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,1
3,Adin,Burger Joint,Market,Women's Store,Food Service,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,1
4,Agoura Hills,Home Service,Thai Restaurant,Women's Store,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,4


In [107]:
neighborhoodCaliforniaFinal.reset_index(inplace=True)
neighborhoodCaliforniaFinal

,Neighborhood,Latitude,Longitude
0,Acampo,38.174640,-121.278559
1,Acton,34.480742,-118.186838
2,Adelaide,37.787483,-122.412150
3,Adelanto,34.582770,-117.409215
4,Adin,41.193780,-120.945245
5,Agoura Hills,34.136395,-118.774535
6,Agua Dulce,34.496382,-118.325635
7,Aguanga,33.442809,-116.865024
8,Ahwahnee,37.367705,-119.725409
9,Alamo,37.850203,-122.032184


In [108]:

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhood_merged = neighborhood_merged.join(neighborhoodCaliforniaFinal.set_index('Neighborhood'), on='Neighborhood')

#neighborhood_merged = neighborhood_merged.dropna()
#neighborhood_merged

In [112]:
neighborhood_merged.dropna(inplace = True)

In [113]:
neighborhood_merged.shape

(523, 14)

## Representing the 5 Clusters on map

In [114]:
map_clusters = folium.Map(location=[34, -118], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_merged['Latitude'], neighborhood_merged['Longitude'], neighborhood_merged['Neighborhood'], neighborhood_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 5

In [115]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 4, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
4,Home Service,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,4,34.136395,-118.774535
66,Home Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,4,35.652175,-118.322581
87,Home Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,4,32.664798,-117.015962
104,Home Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,4,34.144664,-118.644097
321,Home Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,4,38.610516,-122.881341
331,Home Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,4,34.164091,-118.657837
332,Home Service,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,4,34.128344,-117.208651
333,Home Service,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,4,37.552726,-122.330695
460,Home Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,4,33.860072,-117.239877


## Cluster 4

In [116]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 3, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
0,Winery,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,3,38.174640,-121.278559
70,Business Service,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,3,40.002107,-121.306633
80,Business Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,3,39.412388,-121.712751
166,Business Service,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,3,39.264895,-123.591122
194,Business Service,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,3,38.681848,-120.823269
203,Business Service,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,3,38.679827,-120.821194
275,Business Service,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,3,36.938163,-121.802056


## Cluster 3

In [117]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 2, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
139,Grocery Store,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,2,38.533412,-123.085463
188,Grocery Store,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,2,34.863417,-116.888032
217,Grocery Store,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,2,32.644225,-116.781409
324,Grocery Store,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,2,38.295747,-121.244392
369,Grocery Store,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,2,38.965089,-122.837168
433,Grocery Store,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,2,37.271054,-122.308581
457,Grocery Store,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,2,38.970185,-123.688073


## Cluster 2

In [118]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 1, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
1,Hotel,Speakeasy,Art Gallery,Thai Restaurant,Indian Restaurant,Hostel,Spa,1,37.787483,-122.412150
2,Business Service,Concert Hall,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,1,34.582770,-117.409215
3,Burger Joint,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,1,41.193780,-120.945245
5,Bakery,Convenience Store,Grocery Store,Café,Food Court,Fish Market,Flea Market,1,34.496382,-118.325635
6,Intersection,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,1,33.442809,-116.865024
7,Sports Bar,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,1,37.367705,-119.725409
9,Sandwich Place,American Restaurant,Optical Shop,Laundromat,Bagel Shop,Shopping Mall,Chinese Restaurant,1,37.850203,-122.032184
10,Pizza Place,Thai Restaurant,Japanese Restaurant,Sushi Restaurant,French Restaurant,Burger Joint,Café,1,37.886870,-122.297748
11,Campground,Food Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,1,39.223512,-123.768625
12,Ice Cream Shop,Sushi Restaurant,Seafood Restaurant,Café,Movie Theater,Lounge,Market,1,34.093042,-118.127060


## Cluster 1

In [119]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 0, neighborhood_merged.columns[[1] + list(range(5, neighborhood_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
22,Park,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,0,38.174918,-122.260804
76,Bed & Breakfast,Food Truck,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,0,34.267886,-116.848598
144,Home Service,Café,Women's Store,Food Court,Fish & Chips Shop,Fish Market,Flea Market,0,33.864429,-118.053932
179,Sporting Goods Shop,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,0,37.216492,-121.739417
205,Park,Food Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,0,37.914774,-121.594005
221,Hotel,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,0,39.206009,-120.837717
246,Baseball Stadium,Food Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,0,38.694591,-122.021368
247,National Park,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,0,34.733610,-115.244979
248,Bakery,Brewery,Disc Golf,Football Stadium,Flea Market,Flower Shop,Fondue Restaurant,0,41.456807,-122.894755
258,Liquor Store,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,0,35.178418,-119.546120


## Analysis of Clusters

### Cluster 1

Cluster 1 represents location that are near to Park, Church, National Park, Stadium that is good for adventure loving people.

### Cluster 2

Cluster 2 represents location related to mexican restaurant, Fast food chains, bar that is it is good for gourmand people or those who wants to open a food restaurant. 

### Cluster 3

Cluster 3 represents locations that are near to Grocery store or market that is good for those who want to store near themselves, open stores or to sell their porduct in market.

### Cluster 4

Cluster 4 represent place where most of people use business services so if someone wants to setup business services for other businesses it is the right choice.

### Cluster 5

Cluster 5 represent place where most of people use home services so if someone wants to setup home services or shops it is the right choice.